In [1]:
from random import choice
import plotly.graph_objects as go
from operator import add
%run fec.ipynb

In [22]:
MAX_FLOOR = 15
MAX_LOAD = 15
T_F = 2
T_P = 5
NUM_OF_PS = 30

In [13]:
class elevator :
    
    def __init__(self) :
        self.pos = 1
        self.load = 0
        self.ps = []
        self.pps = []
        self.direct = 'none'  # none / up / down
        self.uti = 0
        self.stat = 'idle'  # idle / drive / stop
        self.totalUti = 0
    
    def __str__(self) :
        return (
            f'pos = {self.pos}, load = {self.load}, '
            f'len(ps) = {len(self.ps)}, len(pps) = {len(self.pps)}, '
            f'direct = {self.direct}, uti = {self.uti}, stat = {self.stat}'
        )
    
    def reset(self) :
        self.__init__()
    
class passenger :
    
    def __init__(self, s, d, l) :
        self.src = s
        self.dest = d
        self.load = l
        self.wt = 0
        self.rt = 0
        
    def __str__(self) :
        return (
            f'(s, d, l) = ({self.src}, {self.dest}, {self.load}), '
            f'waiting time = {self.wt}, riding time = {self.rt}'
        )

In [23]:
elevators = [elevator(), elevator()]
passengers = []
served_ps = 0
usingFec = True

def simulate(useFec = True, debug = False) :
    #reset
    for ele in elevators :
        ele.reset()
    passengers.clear()
    global served_ps, usingFec
    served_ps = 0
    usingFec = useFec
    
    #start
    time = 0
    psfreq = 10
    total_ps = 0
    while served_ps < NUM_OF_PS and time < 999:
        if time % psfreq == 0 and total_ps < NUM_OF_PS:
            if not debug :
                ps2hall()
            else :
                ps2hall_T(total_ps)
            total_ps += 1
            
        check()
        plusTime()
        
        #print(f'after chect in time {time}')
        #print(f'elevator 0 : {elevators[0]}')
        #print(f'elevator 1 : {elevators[1]}\n')
        
        time += 1
    
    time += max(elevators[0].uti, elevators[1].uti)
    
    wt_list = []
    rt_list = []
    for ps in passengers :
        wt_list.append(ps.wt)
        rt_list.append(ps.rt)
    meanWt = sum(wt_list) / len(wt_list)
    meanRt = sum(rt_list) / len(rt_list)
    lo_0 = elevators[0].totalUti / time
    lo_1 = elevators[1].totalUti / time
    
    rtn = [True,[meanRt,meanWt,time,lo_0,lo_1]] if served_ps == NUM_OF_PS else [False,[]]
    return rtn
    #print(f'\nsimulation complete, served {served_ps} in {time}(s)')
    #print(f'elevator_0 total uti = {elevators[0].totalUti}')
    #print(f'elevator_1 total uti = {elevators[1].totalUti}')
    
def ps2hall() :
    #populations
    src_pl =  list(range(1, 16)) + [1, 1, 1, 3, 4]
    dest_pl = list(range(1, 16)) + [1, 1, 1, 1, 1, 3, 4]
    load_pl = list(range(1, 7)) + [1, 2, 2, 2, 3, 3, 4]
    src = choice(src_pl)
    dest = choice(dest_pl)
    while dest == src :
        dest = choice(dest_pl)
    load = choice(load_pl)
    ps = passenger(src, dest, load)
    passengers.append(ps)
    
    prio_0 = evalp(elevators[0], ps) if usingFec else td_eval(elevators[0], ps)
    prio_1 = evalp(elevators[1], ps) if usingFec else td_eval(elevators[1], ps)
    if prio_0 > prio_1 :
        elevators[0].pps.append(ps)
    elif prio_0 < prio_1 :
        elevators[1].pps.append(ps)
    else :
        which = choice([0, 1])
        elevators[which].pps.append(ps)
    
def evalp(ele, pa) :
    prio = 0
    if ele.direct == 'up' :
        if ele.pos > pa.src :
            prio -= 1
        elif ele.pos < pa.src :
            no_ps = True if len(ele.ps) == 0 else False
            no_pps = True if len(ele.pps) == 0 else False
            if no_ps and not no_pps :
                fpps = ele.pps[0]
                if fpps.src <= pa.src and fpps.dest < pa.src :
                    prio -= 0.7
            if not no_ps and not no_pps :
                mPsDest = 0
                for ps in ele.ps :
                    mPsDest = max(mPsDest, ps.dest)
                d = True if mPsDest < pa.src else False
                # no passenger on the ele whose dest >= pa.src
                if d :
                    dd = True
                    for pps in ele.pps :
                        if pps.src in range(ele.pos, mPsDest) and pps.dest >= pa.src :
                            dd = False
                    if dd :
                        prio -= 0.7      
    elif ele.direct == 'down' :
        if ele.pos < pa.src :
            prio -= 1
        elif ele.pos > pa.src :
            no_ps = True if len(ele.ps) == 0 else False
            no_pps = True if len(ele.pps) == 0 else False
            if no_ps and not no_pps :
                fpps = ele.pps[0]
                if fpps.src >= pa.src and fpps.dest > pa.src :
                    prio -= 0.7
            if not no_ps and not no_pps :
                mPsDest = 16
                for ps in ele.ps :
                    mPsDest = min(mPsDest, ps.dest)
                d = True if mPsDest > pa.src else False
                # no passenger on the ele whose dest <= pa.src
                if d :
                    dd = True
                    for pps in ele.pps :
                        if pps.src in range(mPsDest, ele.pos) and pps.dest <= pa.src :
                            dd = False
                    if dd :
                        prio -= 0.7 
            
    wt = abs(ele.pos - pa.src) * T_F
    rt = abs(pa.src - pa.dest) * T_F
    ld = ele.load
    td = abs(pa.src - pa.dest)
    hcaw = pa.src
    daw = pa.dest
    prio += fec_eval([wt, rt, ld, td, hcaw, daw])
    
    return prio

def td_eval(ele, pa) :
    prio = 0
    if ele.stat == 'idle' :
        prio += 1
    if ele.direct == 'up' and pa.dest > pa.src :
        prio += 1
    elif ele.direct == 'down' and pa.dest < pa.src :
        prio += 1
    #prio -= abs(ele.pos - pa.src) / MAX_FLOOR
    
    return prio

def check() :
    for ele in elevators :
        if ele.stat == 'drive' or ele.stat == 'stop' :
            ele.uti -= 1
            if ele.uti != 0 :
                continue
            elif ele.stat == 'drive' :
                onFloor(ele)
            elif ele.stat == 'stop' :
                endOfStop(ele)
        if ele.stat == 'idle' :
            onIdle(ele)

def onFloor(ele) :
    if ele.stat == 'drive' :
        if ele.direct == 'up' :
            ele.pos += 1 if ele.pos < MAX_FLOOR else 0
        elif ele.direct == 'down' :
            ele.pos -= 1 if ele.pos > 1 else 0
    
    is_stopping = False
    no_ps = False
    no_pps = False
    # passenger out
    out_ps = []
    for ps in ele.ps :
        if ps.dest == ele.pos :
            out_ps.append(ps)
    for ps in out_ps :
        ele.load -= ps.load
        ele.ps.remove(ps)
        is_stopping = True
        global served_ps
        served_ps += 1
    if len(ele.ps) == 0 :
        no_ps = True
    
    # passenger in
    out_pps = []
    for pps in ele.pps :
        if pps.src == ele.pos and pps.load + ele.load <= MAX_LOAD :
            psd = 'none'
            if pps.src < pps.dest :
                psd = 'up'
            else :
                psd = 'down'
            if not no_ps and psd != ele.direct :
                continue
            ele.ps.append(pps)
            ele.load += pps.load
            if no_ps :
                ele.direct = psd
            out_pps.append(pps)
            is_stopping = True
    for pps in out_pps :
        ele.pps.remove(pps)

    if len(ele.ps) != 0 :
        no_ps = False
    if len(ele.pps) == 0 :
        no_pps = True
    
    if is_stopping :
        ele.stat = 'stop'
        ele.uti = T_P
        ele.totalUti += T_P
        if no_ps :
            ele.direct = 'none'
    else :
        ele.uti = T_F
        ele.totalUti += T_F

def endOfStop(ele) :
    if ele.direct == 'none' :
        ele.stat = 'idle'
    else :
        ele.stat = 'drive'
        ele.uti = T_F
        ele.totalUti += T_F
        
def onIdle(ele) :
    keep_idle = True
    is_stopping = False
    
    out_pps = []
    for pps in ele.pps :
        if pps.src == ele.pos :
            if pps.load + ele.load <= MAX_LOAD :
                psd = 'none'
                if pps.src < pps.dest :
                    psd = 'up'
                else :
                    psd = 'down'
                if ele.direct != 'none' and psd != ele.direct :
                    continue
                ele.ps.append(pps)
                ele.load += pps.load
                if ele.direct == 'none' :
                    ele.direct = psd
                out_pps.append(pps)
                keep_idle = False
                is_stopping = True
        else :
            where2go = 'none'
            if pps.src < ele.pos :
                where2go = 'down'
            else :
                where2go = 'up'
            if ele.direct == 'none' :
                ele.direct = where2go
            keep_idle = False
    for pps in out_pps :
        ele.pps.remove(pps)
    
    if not keep_idle :
        if is_stopping :
            ele.stat = 'stop'
            ele.uti = T_P
            ele.totalUti += T_P
        else :
            ele.stat = 'drive'
            ele.uti = T_F
            ele.totalUti += T_F

def plusTime() :
    for ele in elevators :
        for ps in ele.ps :
            ps.rt += 1
        for pps in ele.pps :
            pps.wt += 1

In [5]:
# ps2hall_T use for debug
def ps2hall_T(t) :
    src = [6,9,10,2,13]
    dest = [12,7,13,6,7]
    load = [3,3,2,2,4]
    ps = passenger(src[t], dest[t], load[t])
    passengers.append(ps)
    prio_0 = evalp(elevators[0], ps)
    prio_1 = evalp(elevators[1], ps)
    if prio_0 >= prio_1 :
        elevators[0].pps.append(ps)
    else :
        elevators[1].pps.append(ps)

In [ ]:
if __name__ == '__main__' :
    #simulat K times
    K = 100
    count = 0
    mwt_list = []
    mrt_list = []
    time_list = []
    lo_0_list = []
    lo_1_list = []
    x = list(range(1, K + 1))
    while count < K :
        result = simulate()
        # if some bugs happened QAQ
        if not result[0] :
            continue
        
        count += 1
        data = result[1]
        mwt = data[0]; mrt = data[1]
        time = data[2]
        lo_0 = data[3]; lo_1 = data[4]
        print(
            f'complete ({count}/{K}) simulations\n'
            f'(mean_wt, mean_rt, total_time, uti_rate_0, uti_rate_1) = '
            f'({mwt:.3f}, {mrt:.3f}, {time}, {lo_0:.3f}, {lo_1:.3f})'
        )
        mwt_list.append(mwt)
        mrt_list.append(mrt)
        time_list.append(time)
        lo_0_list.append(lo_0)
        lo_1_list.append(lo_1)
        
    #make graphs
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x = x, y = mwt_list, name = 'mean waiting time'));
    fig1.add_trace(go.Scatter(x = x, y = mrt_list, name = 'mean riding time'));
    fig1.add_trace(go.Scatter(x = x, y = list(map(add, mwt_list, mrt_list)),
                              name = 'mean Ws(mwt + mrt)'));
    fig1.update_layout(title = 'WT, RT and Ws',
                       xaxis_title = 'K-th simulation',
                       yaxis_title = 'time(s)');
    box1 = go.Figure()
    box1.add_trace(go.Box(y = mwt_list, name = 'mean waiting time'));
    box1.add_trace(go.Box(y = mrt_list, name = 'mean riding time'));
    box1.add_trace(go.Box(y = list(map(add, mwt_list, mrt_list)),
                          name = 'mean Ws(mwt + mrt)'));
    box1.update_layout(title = 'WT, RT and Ws',
                       yaxis_title = 'time(s)');
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x = x, y = time_list, name = 'total time'));
    fig2.update_layout(title = f'total time for {NUM_OF_PS} passenger groups',
                       xaxis_title = 'K-th simulation',
                       yaxis_title = 'time(s)');
    box2 = go.Figure()
    box2.add_trace(go.Box(y = time_list, name = 'total time'));
    box2.update_layout(title = f'total time for {NUM_OF_PS} passenger groups',
                       yaxis_title = 'time(s)');
    
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x = x, y = lo_0_list,
                              name = 'elevator_0\'s uti-rate'));
    fig3.add_trace(go.Scatter(x = x, y = lo_1_list,
                              name = 'elevator_1\'s uti-rate'));
    fig3.update_layout(title = 'uti-rate of elevators',
                       xaxis_title = 'K-th simulation',
                       yaxis_title = 'uti-rate');
    box3 = go.Figure()
    box3.add_trace(go.Box(y = lo_0_list,
                          name = 'elevator_0\'s uti-rate'));
    box3.add_trace(go.Box(y = lo_1_list,
                          name = 'elevator_1\'s uti-rate'));
    box3.update_layout(title = 'uti-rate of elevators',
                       yaxis_title = 'uti-rate');
    
    fig1.show()
    box1.show()
    fig2.show()
    box2.show()
    fig3.show()
    box3.show()